In [1]:
import sys
from vtk import *

In [48]:
pointcloud = "gdcanyon_cloud.vtk"
topography = "gdcanyon_image.vtk"
#read pointcloud
preader = vtk.vtkDataSetReader()
preader.SetFileName(pointcloud)
preader.ReadAllVectorsOn()
preader.ReadAllScalarsOn()
preader.Update()
pmapper = vtkDataSetMapper()

# read topograpy
treader = vtk.vtkDataSetReader()
treader.SetFileName(topography)
treader.ReadAllVectorsOn()
treader.ReadAllScalarsOn()
treader.Update()
tmapper = vtkDataSetMapper()




In [49]:
pout = preader.GetPolyDataOutput()
tout = treader.GetPolyDataOutput()

In [50]:
# delaunay triangulation

#create class
delaunay = vtk.vtkDelaunay2D()
# get input data from point cloud
delaunay.SetInputData(preader.GetPolyDataOutput())
delaunay.Update()

# map mesh
meshMapper = vtk.vtkPolyDataMapper()
meshMapper.SetInputConnection(delaunay.GetOutputPort())

meshActor = vtk.vtkActor()
meshActor.SetMapper(meshMapper)
meshActor.GetProperty().SetEdgeColor(0, 0, 1)
meshActor.GetProperty().SetInterpolationToFlat()
meshActor.GetProperty().SetRepresentationToWireframe()

boundaryMapper = vtk.vtkPolyDataMapper()

boundaryActor = vtk.vtkActor()
boundaryActor.SetMapper(boundaryMapper)
boundaryActor.GetProperty().SetColor(1, 0, 0)

renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
renderer.AddActor(meshActor)
renderer.AddActor(boundaryActor)
renderer.SetBackground(.3, .6, .3)

renderWindowInteractor.Initialize()
renderWindow.Render()
renderWindowInteractor.Start()

In [56]:
# Mesh Free


locator = vtk.vtkStaticPointLocator()
locator.SetDataSet(output)
locator.BuildLocator()


NPts = 100
math = vtk.vtkMath()

#warp function for height map

luminance = vtk.vtkImageLuminance()
luminance.SetInputConnection(treader.GetOutputPort())
geometry = vtk.vtkImageDataGeometryFilter()
geometry.SetInputConnection(luminance.GetOutputPort())
warp = vtk.vtkWarpScalar()
warp.SetInputConnection(geometry.GetOutputPort())
warp.SetNormal(0, 0, 1)
warp.UseNormalOn()
warp.SetScaleFactor(2)
warp.Update()


bounds = preader.GetOutput().GetBounds()
center = preader.GetOutput().GetCenter()


# create point cloud
points = vtk.vtkPoints()
points.SetDataTypeToFloat()
points.SetNumberOfPoints(NPts)

for i in range(0,NPts):
    points.SetPoint(i,math.Random(bound[0],bound[1]),math.Random(bound[2],bound[3]),math.Random(bound[4],bound[5]))



# source for interpolator

source = vtk.vtkPolyData()
source.SetPoints(points)

sphere = vtk.vtkSphere()
sphere.SetCenter(center[0],center[1]-7500,center[2])

attr = vtk.vtkSampleImplicitFunctionFilter()
attr.SetInputData(source)
attr.SetImplicitFunction(sphere)
attr.Update()


shepardKernel = vtk.vtkShepardKernel()
shepardKernel.SetPowerParameter(2)
shepardKernel.SetRadius(0.5)

interpolator = vtk.vtkPointInterpolator()
interpolator.SetInputConnection(points)
interpolator.SetSourceData(pout)
interpolator.SetKernel(shepardKernel)
interpolator.Update()


intmap = vtk.vtkPolyDataMapper()
intmap.SetInputConnection(interpolator.GetOutputPort())

intact = vtk.vtkActor()
intact.SetMapper(intmap)


outline = vtk.vtkStructuredGridOutlineFilter()
outline.SetInputConnection(warp.GetOutputPort())

outlineMapper = vtk.vtkPolyDataMapper()
outlineMapper.SetInputConnection(outline.GetOutputPort())

outlineActor = vtk.vtkActor()
outlineActor.SetMapper(outlineMapper)

renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
renderer.AddActor(outlineActor)
renderer.AddActor(intact)

renderer.SetBackground(.3, .6, .3)

cam = renderer.GetActiveCamera()
cam.SetFocalPoint(center)
fp = cam.GetFocalPoint()
cam.SetPosition(fp[0]+.2,fp[1]+.1,fp[2]+1)
renderer.ResetCamera()


renderWindowInteractor.Initialize()
renderWindow.Render()
renderWindowInteractor.Start()



TypeError: SetInputConnection argument 1: method requires a vtkAlgorithmOutput, a vtkPoints was provided.

In [30]:
# create actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Create a renderer and add the actor to the scene
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)

# Create render window 
renderWindow = vtk.vtkRenderWindow()
renderWindow.SetSize(900, 600) # Set the window size you want
renderWindow.AddRenderer(renderer)

# Set-up interactor
renderWindowInteractor = vtk.vtkRenderWindowInteractor()

# Use track-ball interaction style instead of joystick style
style =	vtk.vtkInteractorStyleTrackballCamera()

renderWindowInteractor.SetInteractorStyle(style)
renderWindowInteractor.SetRenderWindow(renderWindow)

# Render and interact
renderWindow.Render()
renderWindowInteractor.Start()
